In [1]:
import os

import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
from tensorflow.python.framework import graph_io

## Settings

In [2]:
FROZEN_GDEF_PATH = 'data/frozen.pb' # ADJUST
TRT_GDEF_PATH = 'data/frozen_trt.pb' # ADJUST
OUTPUT_NODE = 'net/fc8/BiasAdd' # ADJUST
MAX_BATCH_SIZE = 1 # ADJUST
MAX_WORKSPACE = 1 << 32 # ADJUST
DATA_TYPE = 'FP16' # ADJUST # 'FP16' | 'FP32'

## Load frozen TF graph

In [3]:
graphdef_frozen = tf.GraphDef()
with tf.gfile.GFile(FROZEN_GDEF_PATH, "rb") as f:
    graphdef_frozen.ParseFromString(f.read())

## Build new graph with optimized TensorRT nodes

In [4]:
graphdef_trt = trt.create_inference_graph(
    input_graph_def=graphdef_frozen,
    outputs=[OUTPUT_NODE],
    max_batch_size=MAX_BATCH_SIZE,
    max_workspace_size_bytes=MAX_WORKSPACE,
    precision_mode=DATA_TYPE)

INFO:tensorflow:Running against TensorRT version 5.0.0


## Save new TensorRT graph

In [5]:
os.makedirs(os.path.dirname(TRT_GDEF_PATH), exist_ok=True)
graph_io.write_graph(graphdef_trt, './', TRT_GDEF_PATH, as_text=False)

'./data/frozen_trt.pb'

## List new graph nodes

In [6]:
[x.name for x in graphdef_trt.node]

['net/input',
 'net/Sub/y',
 'net/Reshape/shape',
 'net/fc8/biases',
 'PermConstNHWCToNCHW-LayoutOptimizer',
 'PermConstNCHWToNHWC-LayoutOptimizer',
 'net/Sub',
 'net/conv1_1/Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer',
 'net/pool5/MaxPool-0-0-TransposeNCHWToNHWC-LayoutOptimizer',
 'net/Reshape',
 'net/fc8/BiasAdd',
 'net/my_trt_op_0',
 'net/my_trt_op_1']

## Export new graph for visualization in Tensorboard

In [7]:
graph_trt = tf.Graph()
with graph_trt.as_default():
    tf.import_graph_def(graphdef_trt)
_=tf.summary.FileWriter('data/checkpoints/vggA_BN_frozen_trt/', graph_trt)